In [11]:
#%% General Imports
import pandas as pd
import numpy as np
from scipy.optimize import curve_fit
import os
from matplotlib import pyplot as plt

#sklearn imports
import sklearn.linear_model as linear_model
from sklearn.preprocessing import StandardScaler

#pyomo
from pyomo.environ import *


In [12]:
REPO = 'viscosity_liquid_transfer_Pablo'
parent_path = os.getcwd().split(REPO)[0]

In [32]:

def fit_lin(df):
    lin_model = linear_model.LinearRegression()
    x = df.loc[:,['aspiration_rate','dispense_rate']]
    y = df.loc[:,['%error']]
    
    lin_model.fit(x,y)

    a,b = lin_model.coef_.tolist()[0]
    c= lin_model.intercept_.tolist()[0]

    return a,b,c

def time_for_1000(m):
    
    return 1000/m.x1 + 1000/m.x2 + 10


In [35]:
#data set
df = pd.read_csv(parent_path+REPO+'\\Opentrons_experiments\\BOTorch_optimization\\CCF_initialization\\BOTorch_optimization_CCF_Viscosity_std_1275.csv')


df_duplicates =df.where(df.duplicated(['aspiration_rate','dispense_rate'],keep=False)==True).dropna(how='all')
df_incomplete = df.where(df.duplicated(['aspiration_rate','dispense_rate'],keep=False)==False).dropna(how='all')
df_mean = pd.DataFrame(columns= df.columns)
for index,values in df_duplicates.drop_duplicates(['aspiration_rate','dispense_rate']).iterrows():
    if len(df_duplicates.loc[index:index+2]) == len([300,500,1000]):
        mean_error =df_duplicates.loc[index:index+2,'%error'].mean()
        df_duplicates.loc[index,'%error'] = mean_error
        df_duplicates.loc[index, 'volume'] ='mean'+str([300,500,1000])
        df_mean = pd.concat([df_mean,df.loc[index:index+2],df_duplicates.loc[[index]]])
    else:
        df_incomplete = pd.concat([df_incomplete,df_duplicates.loc[index:index+2]]).drop_duplicates()
df_mean = pd.concat([df_mean,df_incomplete])
df_mean = df_mean.reset_index(drop=True)  



df = df_mean.where(df_mean['volume'] == 'mean[300, 500, 1000]').dropna(how='all')# Create a Pyomo model object
model = ConcreteModel()

# Define decision variables
model.x1 = Var(initialize= 1, bounds = (0.1,100))
model.x2 = Var(initialize= 1, bounds = (0.1,100))

model.obj = Objective(rule= time_for_1000, sense=minimize)

# Define constraints'
a,b,c = fit_lin(df)
model.constraints = ConstraintList()
model.constraints.add(expr= a * model.x1 + b * model.x2 + c <= 2.5)
model.constraints.add(expr= a * model.x1 + b * model.x2 + c >= -2.5)
model.constraints.add(expr= model.obj >= 10)


In [36]:
solver = SolverFactory('ipopt')
solver.solve(model, tee=True)
model.pprint()

Ipopt 3.11.1: 

******************************************************************************
This program contains Ipopt, a library for large-scale nonlinear optimization.
 Ipopt is released as open source code under the Eclipse Public License (EPL).
         For more information visit http://projects.coin-or.org/Ipopt
******************************************************************************

NOTE: You are using Ipopt by default with the MUMPS linear solver.
      Other linear solvers might be more efficient (see Ipopt documentation).


This is Ipopt version 3.11.1, running with linear solver mumps.

Number of nonzeros in equality constraint Jacobian...:        0
Number of nonzeros in inequality constraint Jacobian.:        6
Number of nonzeros in Lagrangian Hessian.............:        2

Total number of variables............................:        2
                     variables with only lower bounds:        0
                variables with lower and upper bounds:        2


In [37]:
model.x1.value

11.801527371592773